In [ ]:
# This page is for provisioning an ESP32 GPS device for the wingtip or base-station.  
# This interfaces with the Hanglog3 android app to record the data, which the 
# scripts in this repository can interact with to enable RTK technology


In [26]:
#### esptool.py --port /dev/ttyUSB0 --chip esp32 write_flash -z 0x1000 /home/julian/executables/micropythonbins/esp32-idf4-20200124-v1.12-87-g96716b46e.bin
%esptool esp32 /home/julian/executables/micropythonbins/esp32-idf4-20200701-unstable-v1.12-606-g07f181a21.bin



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7f3f0c053e90, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Executing:
  esptool.py --port /dev/ttyUSB0 --chip esp32 write_flash -z 0x1000 /home/julian/executables/micropythonbins/esp32-idf4-20200701-unstable-v1.12-606-g07f181a21.bin

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: 3c:71:bf:ff:67:7c
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1448848 bytes to 924753...
Wrote 1448848 bytes (924753 compressed) at 0x00001000 in 81.9 seconds (effective 141.5 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [27]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [28]:
help("modules")

__main__          gc                uasyncio/stream   upip_utarfile
_boot             inisetup          ubinascii         upysh
_onewire          machine           ubluetooth        urandom
_thread           math              ucollections      ure
_uasyncio         micropython       ucryptolib        urequests
_webrepl          neopixel          uctypes           uselect
apa106            network           uerrno            usocket
btree             ntptime           uhashlib          ussl
builtins          onewire           uhashlib          ustruct
cmath             sys               uheapq            utime
dht               uarray            uio               utimeq
ds18x20           uasyncio/__init__ ujson             uwebsocket
esp               uasyncio/core     umqtt/robust      uzlib
esp32             uasyncio/event    umqtt/simple      webrepl
flashbdev         uasyncio/funcs    uos               webrepl_setup
framebuf          uasyncio/lock     upip              websocket_hel

In [29]:
%sendtofile --source utils.py
%sendtofile --source ubxm8t.py

Sent 66 lines (2004 bytes) to ubxm8t.py.


In [477]:
%sendtofile config.txt

deviceletter   C
pinled         2
pingled        23
pinbatteryadc  34,6800,2200
connection1    BV6000,beckaaaa,192.168.43.1,9042
connection0    JulianS9,beckaaaa,192.168.43.1,9042
sdcard         5,18,23,19,/sd
wifi2sdtimeout 10

Sent 8 lines (227 bytes) to config.txt.


In [476]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [50]:
%sendtofile --source main.py

Sent 105 lines (3616 bytes) to main.py.


In [47]:
%sendtofile --source utils.py

Sent 94 lines (2645 bytes) to utils.py.


In [ ]:
if not si.isconnected():
fdatebasedname = "%s%s.ubx" % (datermc, fconfig["deviceletter"])

        

In [ ]:
#
# Code below is for developing and testing the components
#

In [548]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [559]:
%sendtofile --source asyncutils.py

Sent 161 lines (5452 bytes) to asyncutils.py.


In [550]:
from asyncutils import fconfig, wificonnect, initUBX, mountsd, \
                       streamUBX, pinled, flashseq, batteryvoltage


In [551]:
print(batteryvoltage())
mountsd()

3.440918


In [552]:
%ls sd

Listing directory 'sd'.
             sd/OOH/
             sd/System Volume Information/
   133531    sd/hdata-2020-07-06_17-21-04C.ubx
    28170    sd/hdata-2020-07-06_17-24-25C.ubx
    24036    sd/hdata-2020-07-06_20-29-10C.ubx
    85126    sd/hdata-2020-07-06_20-33-51C.ubx
  1641903    sd/hdata-2020-07-06_20-35-30C.ubx
   100028    sd/hdata-2020-07-07_20-13-17C.ubx
  2001393    sd/hdata-2020-07-07_20-16-57C.ubx
  1474115    sd/hdata-2020-07-07_20-43-28C.ubx
        0    sd/test.txt


In [498]:
os.remove("sd/hdata-2020-07-06_17-18-26C.ubx")

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: [Errno 2] ENOENT


In [579]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [581]:
%sendtofile main.py

import uasyncio as asyncio
import uasyncio.stream
from asyncutils import fconfig, wificonnect, initUBX, mountsd, \
                       streamUBX, encodeUBX, pinled, flashseq, batteryvoltage
import socket, os

deviceletter = fconfig["deviceletter"]

fdatebasedname = None
sddir = None
sdfileout = None
nbytesout = 0

androidipnumber, portnumber = None, None
socketstream = None
socketstreamcmd = None

battvoltmsg = None
async def ledflashingtask():
    global battvoltmsg
    lnbytesout = 0
    while sdfileout is None and socketstream is None:
        await asyncio.sleep_ms(1000)
    n = 0
    while True:
        battvolt = batteryvoltage()
        if battvolt != 0 and (n%10) == 0:
            battvoltmsg = encodeUBX(0x21, 0x04, ("battvolt=%.2fv" % battvolt).encode())  # UBX-LOG-STRING
        battfac = max(0.1, battvolt - 6)/4
        if nbytesout - lnbytesout > 1000:
            if sdfileout is not None:
                await flashseq((3000, 100, 200, 100, 200, int(battfac*800), 300, 100, 200, 100))
            else:
                await flashseq((3000, 100, 200, int(battfac*800), 300, 100, ))
                await flashseq((3000, 100, 200, int(battfac*800), 300, 100, ))
            lnbytesout = nbytesout
        else:
            await flashseq((4000, int(battfac*800)))
        n += 1
    
async def manageconnectiontask():
    global socketstream, sdfileout, socketstreamcmd, sddir
    global androidipnumber, portnumber
    androidipnumber, portnumber = await wificonnect()
    if androidipnumber is not None:
        while True:
            try:
                if socketstream is None:
                    ss = socket.socket()
                    ss.settimeout(1)
                    print("new socket connection", ss)
                    ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
                    lsocketstream = uasyncio.stream.Stream(ss)
                    print(await lsocketstream.readline())
                    await lsocketstream.awrite(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
                    if socketstreamcmd is None:
                        socketstream = lsocketstream
                    else:
                        socketstreamcmd = lsocketstream  # drop back into cmd stream on reconnect
                
                while socketstream is not None:
                    print("await socketstream.readline")
                    rl = await socketstream.readline()
                    print("socketstream line", [rl])
                    if rl == b"-SDMODE\n":
                        print("Entering SDMODE")
                        socketstreamcmd, socketstream = socketstream, None
                        break
                        
                # SDMODE here on
                while True:
                    if sddir is None:
                        sddir = mountsd()
                    print("await socketstreamcmd.readline()")
                    rl = (await socketstreamcmd.readline()).decode().strip()
                    print("socketstreamcmd line1", [rl])
                    rld = (await socketstreamcmd.readline()).decode().strip()
                    print("socketstreamcmd line2", [rld])
                    if rl == "-DRL":
                        print("listing")
                        await socketstreamcmd.awrite(("List '%s'\n" % rld).encode())
                        for l in os.listdir(rld):
                            nbytes = os.stat("%s/%s"%(rld,l))[6]
                            await socketstreamcmd.awrite(("%s %d\n" % (l, nbytes)).encode())
                        await socketstreamcmd.awrite((".\n").encode())
                    if rl == "-DRR":
                        print("reading", rld)
                        print(os.stat(rld))
                        nbytes = os.stat(rld)[6]
                        await socketstreamcmd.awrite(("%d\n" % nbytes).encode())
                        fin = open(rld, "rb")
                        while True:
                            d = fin.read(500)
                            if not d:
                                break
                            await socketstreamcmd.awrite(d)
                    if rl == "-DRE":
                        print("erasing", rld)
                        try:
                            os.remove(rld)
                            await socketstreamcmd.awrite(("Removed '%s'\n"%rld).encode())
                        except OSError:
                            await socketstreamcmd.awrite(("Failed\n").encode())
                
            except OSError as e:
                print("socketstream2 OSError", e)
                socketstream = None
                await asyncio.sleep_ms(2000)
                
    # no wifi found, and timeout set, drop into SD card plotting 
    else:
        sddir = mountsd()
        if sddir is not None:
            sdfilename = "%s/%s%s.ubx" % (sddir, fdatebasedname, deviceletter)
            print("Opening sdfile", sdfilename)
            sdfileout = open(sdfilename, "wb")
            flashseq((100, 50, 100, 50, 1000, 200, 1000, 200, 1000))
        else:
            loop.create_task(manageconnectiontask())
        
async def streamgpstask():
    global fdatebasedname, socketstream, nbytesout, battvoltmsg
    fdatebasedname = await initUBX()
    while True:
        fdata = await streamUBX.read(-1)
            
        if socketstream is not None:
            try:
                if battvoltmsg is not None and fdata[:4] == b'\xb5\x62\x01\x30':
                    await socketstream.awrite(battvoltmsg)
                await socketstream.awrite(fdata)
            except OSError as e:
                print("socketstream1 OSError", e)
                socketstream = None  # should kick off connecting again
                
        if sdfileout is not None:
            if battvoltmsg is not None and fdata[:4] == b'\xb5\x62\x01\x30':
                sdfileout.write(battvoltmsg)
            sdfileout.write(fdata)
            sdfileout.flush()

        if battvoltmsg is not None and fdata[:4] == b'\xb5\x62\x01\x30':
            print("ffff", len(fdata), battvoltmsg)
            battvoltmsg = None

        if sdfileout is not None or socketstream is not None:
            np, nbytesout = nbytesout, nbytesout+len(fdata)
            if np//100000 != nbytesout//100000:
                print("bytes:", nbytesout)

loop = asyncio.get_event_loop()
loop.create_task(manageconnectiontask())
loop.create_task(streamgpstask())
loop.create_task(ledflashingtask())
loop.run_forever()

    

Sent 157 lines (6567 bytes) to main.py.


In [438]:
%ls sd

Listing directory 'sd'.
             sd/OOH/
             sd/System Volume Information/
        0    sd/hdata-2020-07-06_17-18-26C.ubx
   133531    sd/hdata-2020-07-06_17-21-04C.ubx
    28170    sd/hdata-2020-07-06_17-24-25C.ubx
        0    sd/test.txt


In [259]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [576]:
print(b'\xb5b\x010\x04\x01\x10\x1d\xf1\x18'[5])

1


In [571]:
async def g():
    for i in range(20):
        fdata = await streamUBX.read(-1)
        if fdata[:4] == b'\xb5\x62\x01\x30':
            print(len(fdata), hex(fdata[2]), hex(fdata[3]), fdata[:10])
    
loop.run_until_complete(g())    


296 0x1 0x30 b'\xb5b\x010\x04\x01\x10\x1d\xf1\x18'
120 0x1 0x30 b'\xb5b\x010\x04\x01p9\xf2\x18'
120 0x1 0x30 b'\xb5b\x010\x04\x018:\xf2\x18'
120 0x1 0x30 b'\xb5b\x010\x04\x01\x00;\xf2\x18'
120 0x1 0x30 b'\xb5b\x010\x04\x01\xc8;\xf2\x18'
120 0x1 0x30 b'\xb5b\x010\x04\x01\x90<\xf2\x18'
